<center>
<H1> Calling a parallel simulation code from Julia</H1>
<figure>
<img src="figures/desman.jpg" width="300" alt="picture of a Galemys pyrenaicus"/>
<figcaption> <it> Galemys pyrenaicus</it> , a mammal endemic from the Pyrenees, where the author lives
</figcaption
</figure>
    <H4> Marc Fuentes, INRIA </H4>
    <H5> code and notebook available at http://github.com/aitzkora/OptimizeMPI.jl
</center>    

## Rationale
 
Imagine you are a Ph.D student and use every a parallel software for simulating some physical process 
relative to your research work; For some reason, you need to optimize some 
physical parameters : In other words, you have to solve an *inverse problem*, whereas 
running your simulation is generally called *direct problem*. 
Good optimization methods are numerous, but implementing state-of-the-art methods in a low-level 
language such as Fortran, C or C++ is not straightforward. In this poster, we want to present 
some recipes (technical and numerical), to do that with "Julia". The knowledge level remains basic
in a perspective to be used as a tutorial . Our hypotheses are the following : 

 - distributed memory paradigm for parallelism (i.e. MPI) 
 - direct problem is Fortran or C.
 - For optimization methods needing a gradient, implementing gradient computation method will be
 done in the low level language.

To sum up with a small sketch it corresponds to 
<img src="figures/rational_call.svg" width="200"/>

## technical tools

To manage to implement our optimization process, we need mainly of two ingredients
1. calling a piece of external code 
2. running julia scripts in a MPI environment 

Let's go : 

### calling Fortran or C code

- to call a external piece of code,  we will use the *Julia* statement `ccall`. As documented
[here](https://docs.julialang.org/en/v1/base/c/) or [there](https://craftofcoding.wordpress.com/2017/02/08/calling-c-from-julia-i-simple-arrays/), the syntax is the following

```julia
ccall((:funcName, library), returnType, (argType1, argType2, ...), (argVal1, argVal2, ...))
ccall(funcPtr, returnType, (argType1, argType2, ...), (argVal1, argVal2, ...))
```

where `function_name` is the mangled name of the C function in the shared library library. If you do not know
what is mangling, take a look at [there](https://en.wikipedia.org/wiki/Name_mangling) : Roughly languages like 
Fortran (due to its case insensitivity) or C++ (in which the same function name could have several 
different signatures), must encode their function names when they interoperate with C.

#### Remarks 
- `library` is only _formally_ a string :
- you could use `"./mylib.so"`   
- but ⚠ you **could not** use `string(pwd(),"/mylib.so")` ⚠ 
- to use a library which is not in `.`, add the path to `LD_LIBRARY_PATH` before launching 
**Julia** (Just tested on Linux, adapt the rule for MacOS with `DYLD_LIBRARY_PATH`)
- using `dlopen` and `dlsym` one could directly use the function pointer call

To start, we could do a small C example, with a function adding 2 to its argument

In [1]:
io = open("/tmp/skel.c","w")
write(io, "int addTwo(int x) { return x+2; }")
close(io)
run(`gcc -o addTwo.so --shared /tmp/skel.c`);
w = ccall((:addTwo, "./addTwo.so"), Int32, (Int32,), 12)
run(`rm addTwo.so /tmp/skel.c`)
println("w = $w")

w = 14


This example deserves some explanations: 
1. to build a *shared* library we add on the gcc compiler command the flag `--shared`. This is evidently,
compiler dependant. If you use Intel, NAG or Microsoft, it could be different. To enforce more portability,
in the sequel, we will use **CMake** as an utility to generate Makefiles and doing the compilation. To
do that, with CMake, one could write
```cmake
add_library(addTwo SHARED /tmp/skel.c)
```
2. One big difference between Fortran and C, is the default argument pass method ; In C, it is by-value,
so a function like `addTwo` cannot modify its arguments. To do that, you need to use pointers and furnish
a Julia **reference** to `ccall`

In [ ]:
io = open("/tmp/skel2.c","w")
write(io, "void addTwoToItsArg(int  * x) { *x= *x+2; }")
close(io)
run(`gcc -o addTwoToItsArg.so --shared /tmp/skel2.c`);
z = Ref{Int32}(12) # note the reference here
w = ccall((:addTwoToItsArg, "./addTwoToItsArg.so"), Cvoid, (Ref{Int32},), z)
run(`rm addTwoItsArg.so /tmp/skel2.c`)
println("z = ",z[])

3. To do the same in Fortran, we can use the following code

```fortran
module example
  use iso_c_binding
contains
  subroutine addTwoF(x) bind(C, name ="addTwoF")
    integer(c_int), intent (inout) :: x
    x = x + 2
  end subroutine 
end module
```

In this example, we used the statement bind to attach a C name to our Fortran function. It will override
the mangled name, when we will use `ccall`. To enforce compatibility, Fortran 90 has some 
C compatibles types , such as  `real(c_double)`, `integer(c_int)` defined in the module `iso_c_binding`.
The `intent(inout)` does not change how the argument is passed (by reference), it is just a information
to enable the compiler to do more checks.

In [4]:
run(`gfortran -o addTwoF.so --shared ./examples/addTwoF.f90`);
z = Ref{Int32}(12) # ✏️ VERY IMPORTANT ✏️
w = ccall((:addTwoF, "./addTwoF.so"), Cvoid, (Ref{Int32},), z)
run(`rm addTwoF.so`)
println("z = ",z[])

z = 14


4. To end with external code calling, we have to speak about arrays : Julia arrays can be convert 
to pointers without any problem, when using `ccall` As an example, 
```C
void changeArray(int n, double * x) { if (n > 1) x[0] += 3 ; }
```

In [2]:

run(`gcc -o changeArray.so --shared examples/changeArray.c`);
a = [1:3.;]
w = ccall((:changeArray, "./changeArray.so"), Cvoid, (Int32, Ptr{Float64},), size(a,1), a)
run(`rm changeArray.so`)
println("a = $a")


a = [4.0, 2.0, 3.0]


### Interacting with MPI

Interacting with MPI, is not so hard, thanks to the good job done by authors of `MPI.jl` and 
`MPIClusterManagers` packages. Perhaps, before to build those packages, you need to select 
the good implementation on your system used by julia setting up some environment variables
in your `startup.jl` file :
```julia
ENV["JULIA_MPI_C_LIBRARIES"] = "-L/usr/lib/openmpi/ -lmpi"
ENV["JULIA_MPI_Fortran_INCLUDE_PATH"] = "-I/usr/include"
ENV["JULIA_MPI_PATH"] = "/usr/bin/mpirun"
```

Using `MPI.jl`, the following "hello world" program

```julia
using MPI
MPI.Init()
println("Hi from $(MPI.Comm_rank(MPI.COMM_WORLD))!")
flush(stdout)
```

could be run directly from shell : the mpi runner calls explicitly Julia

In [1]:
run(`mpirun -np 2 julia examples/hello_world.jl`);

Hi from 0!
Hi from 1!


Doing so, since `mpirun` calls julia, the julia code, is JIT-compiled before to execute each time we run 
the script ; Furthermore, we must run the code out of the Julia REPL, which is not very 
convenient for doing some experiments. To avoid
that, we will use the `MPIClusterManagers` package's macro `@mpi_do`

For example, one could run

In [ ]:
using MPIClusterManagers, Distributed
manager = MPIManager(np=4)
addprocs(manager)
println("Added procs $(procs())")
@everywhere import MPI

and re-run several times the following block without restarting julia

In [ ]:
@mpi_do manager begin 
    comm = MPI.COMM_WORLD
    p = MPI.Comm_size(comm);r = MPI.Comm_rank(comm)
    s_loc=sum(1+r* 100/p:100/p * (r+1)) 
    s = MPI.Reduce(s_loc, +, 0, comm)
    if ( r == 0) 
       println("s=$s")
    end
end

Unfortunately, the present version of MPIClusterManagers does not have an `@mpi_fetchcall` macro
to retrieve the result of the computation on the master (which is not part of the MPI Cluster).
Now we could look at the numerical part of this presentation

### Distributed Minimization

## preliminary remark
Formally, we want to solve a problem like

In [ ]:
%%latex
\arg_{x_1,\cdots,x_p} \min f(x_1,\cdots,x_p) \mbox { where } x_i \in \mathbb{R}^{n_i}

the main function $f$ is assumed decomposable, i.e, we could write as a max or a sum of functions defined
on each  $\mathbb{R}^{n_i}$, for instance

In [ ]:
%%latex
f(x_1,\cdots,x_p) = \sum_{i=1}^p f_i(x_i)

here we plan to use, the package `Optim` to solve our optimization problem. In almost all optimization 
methods, you have a linesearch step, which is responsible to find the step length $\alpha_i^k$ to
do in the current descent direction. Since the computation of $f$ is distributed, each algorithm, 
has its proper version of the current step. Fortunately, if the linesearch use only value 
function, which it is the same on each process, the `p` values of $\alpha_i^k$ will be the same. We must
keep in mind, that if we plan to use algorithm where random strategy is use, it completely destroys
the method.

